In [1]:
import pandas as pd
import spacy
from sklearn import model_selection
import numpy as np


from tensorflow import keras
from tensorflow.keras import layers, models

In [2]:
df = pd.read_json(open("../../data/data.json", "r", encoding="utf8"))

In [3]:
df

,title,author,year,text,themes
0,"[45, 4, 141, 898]",0,2002,"[2, 2329, 1094, 5, 2329, 295, 9741, 1002, 412,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,[2833],1,2002,"[1, 95, 106, 4422, 3, 144, 292, 32, 1425, 8751...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,[209],2,2002,"[106, 19, 717, 21, 1465, 329, 240, 162, 421, 2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[259, 1207]",3,2002,"[10, 648, 76, 139, 541, 206, 679, 179, 107, 44...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,"[522, 522]",4,2001,"[175, 532, 352, 153, 324, 1903, 2272, 1356, 37...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...
17067,"[151, 16, 2, 5151]",163,1990,"[1, 395, 2485, 331, 545, 8900, 28, 2562, 93, 8...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
17068,[],1160,1990,"[2, 596, 152, 155, 200, 277, 167, 4093, 3064, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
17070,"[15, 114]",3182,1990,"[114, 20, 73, 125, 631, 2, 795, 1, 99, 1127, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
17071,"[15, 25, 1, 13]",3425,2005,"[15, 1, 13, 37, 36, 15, 1, 312, 178, 541, 38, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [4]:
# one hot encoding for authors
def encode_authors(author_code):
    qty = df.author.max()
    result = [0] * (qty + 1)
    result[author_code] = 1
    return result

df.author = df.author.apply(encode_authors)

In [5]:
max_features = 10000  # maximum number of words in vocabulari 5000
max_len = 150  # max length of string

In [20]:
df['joined_text'] = df['text'] + df['title']
df['X2'] = df['themes'] + df['author'] 
train_df, test_df = model_selection.train_test_split(df, test_size=0.1, random_state=42)
X1_train = keras.preprocessing.sequence.pad_sequences(train_df['joined_text'].to_list(), maxlen=max_len, padding='post')
X2_train = np.stack(train_df['X2'])
Y_train = np.stack(train_df['year'])
X1_test = keras.preprocessing.sequence.pad_sequences(list(test_df['joined_text']), maxlen=max_len, padding='post')
X2_test = np.stack(test_df['X2'])
Y_test = np.stack(test_df['year'])
#X = keras.preprocessing.sequence.pad_sequences(list(joined_text), maxlen=max_len, padding='post')
#Y = np.array(df['year'].to_list())

In [8]:
#X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=0.1, random_state=42)

### Model 

In [9]:
embedding_dim =64
model1 = keras.models.Sequential([
  keras.layers.Embedding(input_dim=max_features,
                           output_dim=embedding_dim,
                           input_length=max_len),
  keras.layers.Flatten(),
  keras.layers.Dense(2000,activation='relu'),
  keras.layers.Dense(500,activation='relu'),
  keras.layers.Dense(100,activation='relu'),
  keras.layers.Dense(1, activation='relu')
])

model1.compile(optimizer='nadam',
              loss='mean_squared_error',
              metrics=['MAE'])

model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 150, 64)           640000    
_________________________________________________________________
flatten (Flatten)            (None, 9600)              0         
_________________________________________________________________
dense (Dense)                (None, 2000)              19202000  
_________________________________________________________________
dense_1 (Dense)              (None, 500)               1000500   
_________________________________________________________________
dense_2 (Dense)              (None, 100)               50100     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 20,892,701
Trainable params: 20,892,701
Non-trainable params: 0
____________________________________________

In [30]:
epochs = 10
model1.fit(X1_train, Y_train,
          #batch_size=128,
          validation_data=(X1_test,Y_test),
          epochs=epochs)

Epoch 1/10
383/383 [==============================] - 89s 233ms/step - loss: 8556.6201 - MAE: 74.4645 - val_loss: 15109.0928 - val_MAE: 108.6547
Epoch 2/10
383/383 [==============================] - 86s 224ms/step - loss: 5662.6069 - MAE: 58.2148 - val_loss: 20861.9570 - val_MAE: 134.3201
Epoch 3/10
383/383 [==============================] - 87s 228ms/step - loss: 5938.8325 - MAE: 61.7655 - val_loss: 11096.2188 - val_MAE: 84.2841
Epoch 4/10
383/383 [==============================] - 86s 224ms/step - loss: 6859.3418 - MAE: 66.9046 - val_loss: 4678.7656 - val_MAE: 47.7620
Epoch 5/10
383/383 [==============================] - 88s 229ms/step - loss: 4258.4297 - MAE: 51.1605 - val_loss: 8924.2217 - val_MAE: 80.1265
Epoch 6/10
383/383 [==============================] - 86s 223ms/step - loss: 4161.6177 - MAE: 51.2901 - val_loss: 5692.5161 - val_MAE: 57.1186
Epoch 7/10
383/383 [==============================] - 87s 228ms/step - loss: 3727.5210 - MAE: 47.3247 - val_loss: 7022.2104 - val_MAE: 67

In [12]:
score1 = model1.evaluate(np.array(X1_test), np.array(Y_test)) 

print("Test Score:", score1[0])
print("Test Accuracy:", score1[1])

43/43 [==============================] - 1s 15ms/step - loss: 4995.4761 - MAE: 48.5769
Test Score: 4995.47607421875
Test Accuracy: 48.57685089111328


In [ ]:
#X2 = np.array([row.author + row.themes for index, row in df.iterrows()])
#Y2 = np.array(df.year)

In [22]:

model2 = keras.models.Sequential([
  keras.layers.Dense(units=2000, activation='relu', input_shape=[len(X2_train[0])]),
  
  keras.layers.Dense(2000,activation='relu'),
  keras.layers.Dense(500,activation='relu'),
  keras.layers.Dense(100,activation='relu'),
  keras.layers.Dense(1, activation='relu')
])

model2.build()

In [23]:
model2.compile(optimizer='nadam',
              loss='mean_squared_error',
              metrics=['MAE'])

In [24]:
model2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 2000)              8244000   
_________________________________________________________________
dense_10 (Dense)             (None, 2000)              4002000   
_________________________________________________________________
dense_11 (Dense)             (None, 500)               1000500   
_________________________________________________________________
dense_12 (Dense)             (None, 100)               50100     
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 101       
Total params: 13,296,701
Trainable params: 13,296,701
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#X2_train, X2_test, Y2_train, Y2_test = model_selection.train_test_split(X2, Y2, test_size=0.1, random_state=42)


In [25]:
model2.fit(X2_train, Y_train, epochs=5, validation_split=0.1)

Epoch 1/5
345/345 [==============================] - 62s 179ms/step - loss: 184259.6406 - MAE: 237.7271 - val_loss: 121784.2578 - val_MAE: 317.5758
Epoch 2/5
345/345 [==============================] - 65s 189ms/step - loss: 33094.7148 - MAE: 142.8929 - val_loss: 36532.8906 - val_MAE: 165.4467
Epoch 3/5
345/345 [==============================] - 73s 213ms/step - loss: 22335.3809 - MAE: 116.3359 - val_loss: 78095.1250 - val_MAE: 263.3219
Epoch 4/5
345/345 [==============================] - 64s 187ms/step - loss: 16585.0723 - MAE: 100.1085 - val_loss: 55233.3867 - val_MAE: 211.2918
Epoch 5/5
345/345 [==============================] - 65s 189ms/step - loss: 14944.7119 - MAE: 94.5273 - val_loss: 22504.0703 - val_MAE: 123.8234


In [18]:
score2 = model2.evaluate(np.array(X2_test), np.array(Y_test)) 

print("Test Score:", score2[0])
print("Test Accuracy:", score2[1])

43/43 [==============================] - 1s 15ms/step - loss: 37064.2734 - MAE: 172.6725
Test Score: 37064.2734375
Test Accuracy: 172.67250061035156


In [26]:
y_pred1 = model1.predict(X1_test)
y_pred2 = model2.predict(X2_test)
result = (y_pred1 + y_pred2)/2

In [29]:

keras.metrics.mean_absolute_error(
    y_pred1.flatten(), y_pred2.flatten()
)



<tf.Tensor: shape=(), dtype=float32, numpy=122.54742>

In [ ]:
output_dim = 100

inputs = keras.Input(shape=(None,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(max_features, output_dim)(inputs)
x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

# We add a vanilla hidden layer:
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

predictions = layers.Dense(1, activation='relu', name="predictions")(x)

model3 = keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model3.compile(loss="mean_squared_error", optimizer="adam", metrics=["MAE"])


In [ ]:
model3.summary()

In [ ]:
batch_size = 64
epochs = 5
model3.fit(X1_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)

In [ ]:
accr = model3.evaluate(X1_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))